In [36]:
import pathlib
import os
import cv2
from airo_camera_toolkit.utils import ImageConverter
from microplate.opencv_annotation import get_manual_annotations_image , Annotation
import numpy as np
from microplate import PACKAGE_PATH
from microplate.camera_pose import get_camera_pose, pose_model_to_homogeneous_matrix

In [37]:

images_to_label_indices = [0,7,17,-1]
keypoint_names = ["topleft","topright","bottomleft"]
annotation_config = {keypoint: Annotation.Keypoint for keypoint in keypoint_names}

directory = PACKAGE_PATH.parents[1] / "data"/"1"
images = os.listdir(directory)
images = [image for image in images if image.endswith(".png")]
images = sorted(images)
keypoint_dict = {image: {} for image in images}
manually_labeled_keypoints = {images[id]: {} for id in images_to_label_indices}
for image_path, _ in manually_labeled_keypoints.items():
    image = cv2.imread(str(directory / image_path))
    image = ImageConverter.from_opencv_format(image).image_in_numpy_format
    annotation = get_manual_annotations_image(image, annotation_config)
    annotation = {key: np.array(value) for key, value in annotation.items()}
    manually_labeled_keypoints[image_path] = annotation


In [41]:
import json
from airo_dataset_tools.data_parsers.pose import Pose
from microplate.multi_view_triangulation import multiview_triangulation_midpoint

for keypoint in keypoint_names:
    print(keypoint)
    labeled_extrinsics = []
    labeled_intrinsics = []
    labeled_image_points = []
    for image_path, image_points in manually_labeled_keypoints.items():
        extrinsics_path = image_path.replace(".png","_camera_pose.json")
        extrinsics = Pose(**json.load(open(directory/extrinsics_path)))
        extrinsics = pose_model_to_homogeneous_matrix(extrinsics)
        intrinsics_path = image_path.replace(".png","_camera_intrinsics.npy")
        camera_intrinsics = np.load(str(directory/intrinsics_path))
        image_point = np.array(image_points[keypoint])
        labeled_extrinsics.append(extrinsics)
        labeled_intrinsics.append(camera_intrinsics)
        labeled_image_points.append(image_point)

    # triangulate the points
    print(labeled_image_points)
    keypoint_position = multiview_triangulation_midpoint(labeled_extrinsics,labeled_intrinsics,labeled_image_points)
    print(f"{keypoint} position: {keypoint_position}")

    for image_path in keypoint_dict.keys():
        extrinsics_path = image_path.replace(".png","_camera_pose.json")
        extrinsics = Pose(**json.load(open(directory/extrinsics_path)))
        extrinsics = pose_model_to_homogeneous_matrix(extrinsics)
        intrinsics_path = image_path.replace(".png","_camera_intrinsics.npy")
        camera_intrinsics = np.load(str(directory/intrinsics_path))
        image_point = camera_intrinsics @ (np.linalg.inv(extrinsics) @ np.append(keypoint_position,1))[:3]
        image_point = image_point[:2] / image_point[2]
        keypoint_dict[image_path][keypoint] = image_point
        image = cv2.imread(str(directory/image_path))
        cv2.circle(image,(int(image_point[0]),int(image_point[1])),5,(0,0,255),-1)
        # cv2.imshow("image",image)
        # cv2.waitKey(0)
    cv2.destroyAllWindows()


topleft
[array([460, 149]), array([1052,  573]), array([577,  76]), array([654, 607])]
topleft position: [0.31067457 0.07832142 0.01386525]
topright
[array([830, 230]), array([488, 661]), array([880, 295]), array([447, 357])]
topright position: [0.40313697 0.01110004 0.01823323]
bottomleft
[array([347, 331]), array([975, 208]), array([408, 245]), array([825, 464])]
bottomleft position: [0.26805773 0.01622012 0.01215231]


In [42]:
keypoint_dict

{'20230803_182534.png': {'topleft': array([467.23753417, 143.25902777]),
  'topright': array([835.02489864, 223.87494388]),
  'bottomleft': array([347.54017769, 327.53171907])},
 '20230803_182536.png': {'topleft': array([564.0531641 , 109.14919079]),
  'topright': array([884.77955223, 279.56335314]),
  'bottomleft': array([384.38953736, 253.46342524])},
 '20230803_182539.png': {'topleft': array([399.76275917, 114.19745387]),
  'topright': array([836.35430319, 176.35703696]),
  'bottomleft': array([304.49212391, 368.51944747])},
 '20230803_182542.png': {'topleft': array([565.22832244,  40.30167425]),
  'topright': array([923.15944819, 282.66679111]),
  'bottomleft': array([355.66565098, 223.66978723])},
 '20230803_182547.png': {'topleft': array([268.03646632, 176.57733594]),
  'topright': array([769.52975988,  96.00662523]),
  'bottomleft': array([298.80503496, 506.84765477])},
 '20230803_182553.png': {'topleft': array([607.09908463, -37.21143028]),
  'topright': array([961.91471699, 32

In [43]:
for image_path,keypoints in keypoint_dict.items():
    for key,val in keypoints.items():
        keypoints[key] = [int(val[0]),int(val[1])]
        
    keypoint_dict_path = image_path.replace('.png','_keypoints.json')
    with open(directory/keypoint_dict_path, 'w') as fp:
        json.dump(keypoints, fp)